### Geo Foundation Model
https://arxiv.org/pdf/2302.04476

For pretraining, we employ 8 NVIDIA V100 GPUs with
a batch size of 2048 (128 per GPU) and the image size
of 192×192. All pretraining settings are the same as in
[43]. For downstream tasks, 4 NVIDIA A10G GPUs are
employed. During the pretraining stage, we utilize RGB
bands as they are most commonly available among data
sources and tasks.

### nb by looking for maximum in submission file i found that there are 1540x1540 tiles

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
from shapely.geometry import Polygon
import pyproj
import warnings

In [ ]:

# Path to your AOI shapefile
shapefile_path = 'shp_test_AOIs/shp/aoi_2021_04.shp'

# Load the shapefile
aoi = gpd.read_file(shapefile_path)

# Display basic information
print("=== Shapefile Information ===")
print(aoi.info())

# Display the first few rows
print("\n=== First Five Rows ===")
print(aoi.head())

# Print Coordinate Reference System (CRS)
print("\n=== Coordinate Reference System (CRS) ===")
print(aoi.crs)

# Plot the shapefile
aoi.plot(color='lightblue', edgecolor='black')
plt.title('AOI: aoi_2021_04.shp')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.show()

In [ ]:
# Path to one of your AOI shapefiles
# Replace with the desired shapefile
aoi_shapefile = 'shp_test_AOIs/shp/aoi_2021_03.shp'

# Load the shapefile
aoi_gdf = gpd.read_file(aoi_shapefile)

# Inspect the first few rows and columns
print(aoi_gdf.head())
print(aoi_gdf.columns)
print(aoi_gdf)

In [8]:
def lon_to_utm_zone(lon):
    """Calculate the UTM zone number from longitude, capped between 1 and 60."""
    zone = int((lon + 180) / 6) + 1
    zone = min(max(zone, 1), 60)
    return zone

def get_utm_crs(lat, lon):
    """Return the EPSG code for the UTM CRS based on latitude and longitude."""
    zone = lon_to_utm_zone(lon)
    if lat >= 0:
        epsg_code = 32600 + zone  # Northern hemisphere
    else:
        epsg_code = 32700 + zone  # Southern hemisphere
    return f'EPSG:{epsg_code}'

def calculate_aoi_metrics(shp_dir):
    """
    Calculate area, width, and height for each shapefile in the directory.

    Parameters:
    - shp_dir (str): Path to the directory containing shapefiles.

    Returns:
    - pd.DataFrame: DataFrame containing metrics for each AOI.
    """
    # List all .shp files in the directory
    shp_files = [f for f in os.listdir(shp_dir) if f.endswith('.shp')]

    # Initialize a list to store results
    results = []

    # Process each shapefile
    for shp_file in shp_files:
        # Full path to the shapefile
        shp_path = os.path.join(shp_dir, shp_file)
        
        try:
            # Read the shapefile using GeoPandas
            gdf = gpd.read_file(shp_path)
        except Exception as e:
            print(f"Error reading {shp_file}: {e}")
            continue

        # Ensure the GeoDataFrame has a CRS
        if gdf.crs is None:
            print(f"Shapefile {shp_file} has no CRS. Skipping.")
            continue

        # Reproject to WGS84 if not already
        if gdf.crs.to_string() != 'EPSG:4326':
            gdf = gdf.to_crs('EPSG:4326')

        # Select the first geometry (assuming one AOI per shapefile)
        try:
            runway = gdf.iloc[0]
            geometry = runway.geometry
        except Exception as e:
            print(f"Error processing geometry in {shp_file}: {e}")
            continue

        # Calculate centroid
        centroid = geometry.centroid
        lon, lat = centroid.x, centroid.y

        # Get appropriate UTM CRS
        utm_crs = get_utm_crs(lat, lon)

        try:
            # Reproject to UTM CRS for accurate measurements
            gdf_projected = gdf.to_crs(utm_crs)
        except Exception as e:
            print(f"Error reprojecting {shp_file} to {utm_crs}: {e}")
            continue

        # Calculate area in square meters
        try:
            area = gdf_projected['geometry'].area.iloc[0]
        except Exception as e:
            print(f"Error calculating area for {shp_file}: {e}")
            area = None

        # Get the bounding box dimensions (minx, miny, maxx, maxy)
        try:
            minx, miny, maxx, maxy = gdf_projected.total_bounds
            width = maxx - minx  # Width in meters
            height = maxy - miny  # Height in meters
        except Exception as e:
            print(f"Error calculating dimensions for {shp_file}: {e}")
            width = height = None

        # Append the results
        results.append({
            'Shapefile': shp_file,
            'Area (sq m)': round(area, 2) if area else None,
            'Width (m)': round(width, 2) if width else None,
            'Height (m)': round(height, 2) if height else None
        })

    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    return df

if __name__ == "__main__":
    # Directory containing the shapefiles
    shp_dir = 'shp_test_AOIs/shp'  # Update this path if necessary

    # Calculate metrics
    metrics_df = calculate_aoi_metrics(shp_dir)

    # Display the results
    print(metrics_df)

    # Optionally, save to a CSV file
    metrics_df.to_csv('aoi_metrics.csv', index=False)
    print("Metrics have been saved to 'aoi_metrics.csv'.")


FileNotFoundError: [Errno 2] No such file or directory: 'shp_test_AOIs/shp'

In [9]:
def find_geospatial_files(directory, extensions=['.shp', '.geojson']):
    """
    Recursively find all shapefiles and GeoJSON files in a directory.
    Returns a list of file paths.
    """
    geospatial_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                geospatial_files.append(os.path.join(root, file))
    return geospatial_files

def get_shapefile_prj(shp_path):
    """
    Given a shapefile path, return the path to its corresponding .prj file.
    """
    base, _ = os.path.splitext(shp_path)
    prj_path = base + '.prj'
    if os.path.exists(prj_path):
        return prj_path
    else:
        return None

def read_prj_crs(prj_path):
    """
    Read the CRS from a .prj file using pyproj.
    Returns the EPSG code if possible, else returns the full WKT string.
    """
    with open(prj_path, 'r') as prj_file:
        prj_text = prj_file.read()
    try:
        crs = pyproj.CRS.from_wkt(prj_text)
        if crs.to_epsg():
            return f"EPSG:{crs.to_epsg()}"
        else:
            return crs.to_string()
    except Exception as e:
        warnings.warn(f"Unable to parse CRS from {prj_path}: {e}")
        return "Unknown CRS"

def inspect_crs(files, target_crs='EPSG:32718'):
    """
    Inspect and print CRS of given geospatial files.
    For shapefiles, ensure that the .prj file matches the CRS inferred by GeoPandas.
    For GeoJSON files, confirm they are in the expected CRS (default EPSG:4326).
    Warns if discrepancies are found.
    """
    if not files:
        print("No geospatial files found to inspect.")
        return
    
    summary = {
        'Shapefiles': {'Total': 0, 'With PRJ': 0, 'Without PRJ': 0, 'CRS Match': 0, 'CRS Mismatch': 0},
        'GeoJSON': {'Total': 0, 'Expected CRS': 0, 'Different CRS': 0}
    }
    
    print("\n=== CRS Inspection Report ===\n")
    
    for file in files:
        file_lower = file.lower()
        if file_lower.endswith('.shp'):
            summary['Shapefiles']['Total'] += 1
            prj_path = get_shapefile_prj(file)
            print(f"Inspecting Shapefile: {file}")
            
            if prj_path:
                summary['Shapefiles']['With PRJ'] += 1
                prj_crs = read_prj_crs(prj_path)
                print(f"  .prj CRS: {prj_crs}")
            else:
                summary['Shapefiles']['Without PRJ'] += 1
                prj_crs = "Not Defined"
                print("  Warning: Missing .prj file.")
            
            try:
                gdf = gpd.read_file(file)
                gdf_crs = gdf.crs.to_string() if gdf.crs else "Not Defined"
                print(f"  GeoPandas Inferred CRS: {gdf_crs}")
                
                if prj_crs != "Not Defined":
                    if gdf_crs == prj_crs:
                        summary['Shapefiles']['CRS Match'] += 1
                        print("  Status: ✅ CRS Match\n")
                    else:
                        summary['Shapefiles']['CRS Mismatch'] += 1
                        print("  Status: ❌ CRS Mismatch\n")
                        warnings.warn(f"CRS mismatch in {file}: .prj indicates {prj_crs}, but GeoPandas infers {gdf_crs}.")
                else:
                    print("  Status: ❌ CRS Undefined\n")
                    warnings.warn(f"CRS undefined for {file}.")
            
            except Exception as e:
                print(f"  Error reading {file}: {e}\n")
                warnings.warn(f"Error reading {file}: {e}")
        
        elif file_lower.endswith('.geojson'):
            summary['GeoJSON']['Total'] += 1
            print(f"Inspecting GeoJSON: {file}")
            try:
                gdf = gpd.read_file(file)
                gdf_crs = gdf.crs.to_string() if gdf.crs else "Not Defined"
                print(f"  Inferred CRS: {gdf_crs}")
                
                if 'EPSG:4326' == gdf_crs:
                    summary['GeoJSON']['Expected CRS'] += 1
                    print("  Status: ✅ Expected CRS (EPSG:4326)\n")
                else:
                    summary['GeoJSON']['Different CRS'] += 1
                    print("  Status: ❌ Different CRS than Expected (EPSG:4326)\n")
                    warnings.warn(f"GeoJSON {file} has CRS {gdf_crs}, expected EPSG:4326.")
            
            except Exception as e:
                print(f"  Error reading {file}: {e}\n")
                warnings.warn(f"Error reading {file}: {e}")
    
    # Summary Report
    print("\n=== Summary ===\n")
    print("Shapefiles:")
    print(f"  Total Shapefiles: {summary['Shapefiles']['Total']}")
    print(f"  With .prj: {summary['Shapefiles']['With PRJ']}")
    print(f"  Without .prj: {summary['Shapefiles']['Without PRJ']}")
    print(f"  CRS Match: {summary['Shapefiles']['CRS Match']}")
    print(f"  CRS Mismatch: {summary['Shapefiles']['CRS Mismatch']}\n")
    
    print("GeoJSON Files:")
    print(f"  Total GeoJSON Files: {summary['GeoJSON']['Total']}")
    print(f"  Expected CRS (EPSG:4326): {summary['GeoJSON']['Expected CRS']}")
    print(f"  Different CRS: {summary['GeoJSON']['Different CRS']}\n")
    
    # Warnings
    if summary['Shapefiles']['CRS Mismatch'] > 0 or summary['GeoJSON']['Different CRS'] > 0:
        print("Warnings were issued for CRS discrepancies. Please review the warnings above.")
    else:
        print("All inspected files have consistent and expected CRSs.")

def main():
    # Directories to inspect
    training_dirs = ['pac_2024_training/', 'shp_test_AOIs/']
    
    # Find all geospatial files in these directories
    all_files = []
    for directory in training_dirs:
        if os.path.exists(directory):
            print(f"Searching in Directory: {directory}")
            files = find_geospatial_files(directory)
            if files:
                print(f"Found {len(files)} geospatial file(s) in {directory}.\n")
                all_files.extend(files)
            else:
                print(f"No geospatial files found in {directory}.\n")
        else:
            print(f"Directory {directory} does not exist.\n")
    
    # Inspect CRS of all found files
    inspect_crs(all_files)

if __name__ == "__main__":
    main()


Directory pac_2024_training/ does not exist.

Directory shp_test_AOIs/ does not exist.

No geospatial files found to inspect.


In [ ]:
# Check AOI dims both in meters and in longitude/latitude
# Check if test AOIs are square in meters or in longitude/latitude or neither
# TODO make logic nice

for aoi in test_shapefiles:
    aoi_gdf = gpd.read_file(os.path.join(test_shapefiles_dir, aoi))
    aoi_bounds = aoi_gdf.total_bounds
    aoi_width = aoi_bounds[2] - aoi_bounds[0]
    aoi_height = aoi_bounds[3] - aoi_bounds[1]
    print(f"AOI: {aoi}")
    print(f"Width: {aoi_width} meters")
    print(f"Height: {aoi_height} meters")
    print(f"Width: {aoi_width * 111320} degrees longitude")
    print(f"Height: {aoi_height * 111320} degrees latitude")
    print()

AOI: aoi_2021_04.shp
Width: 15410.0 meters
Height: 15270.0 meters
Width: 1715441200.0 degrees longitude
Height: 1699856400.0 degrees latitude

AOI: aoi_2022_01.shp
Width: 15410.0 meters
Height: 15270.0 meters
Width: 1715441200.0 degrees longitude
Height: 1699856400.0 degrees latitude

AOI: aoi_2024_01.shp
Width: 15410.0 meters
Height: 15270.0 meters
Width: 1715441200.0 degrees longitude
Height: 1699856400.0 degrees latitude

AOI: aoi_2020_02.shp
Width: 15410.0 meters
Height: 15270.0 meters
Width: 1715441200.0 degrees longitude
Height: 1699856400.0 degrees latitude

AOI: aoi_2021_01.shp
Width: 15410.0 meters
Height: 15250.0 meters
Width: 1715441200.0 degrees longitude
Height: 1697630000.0 degrees latitude

AOI: aoi_2020_03.shp
Width: 15410.0 meters
Height: 15270.0 meters
Width: 1715441200.0 degrees longitude
Height: 1699856400.0 degrees latitude

AOI: aoi_2020_01.shp
Width: 15410.0 meters
Height: 15270.0 meters
Width: 1715441200.0 degrees longitude
Height: 1699856400.0 degrees latitude
